test new classfier dict

In [1]:
import torch
import dgl
import torch.nn as nn
import dgl.function as fn
from dgl.nn.pytorch.conv import EdgeWeightNorm, GraphConv
import numpy as np

Using backend: pytorch


In [2]:
class Classifier7(nn.Module):
    def __init__(self, arg): 
        super(Classifier7, self).__init__()
        self.n_classes = arg.num_labels
        self.pip_num = 3

        ## // todo: need to rectify to the dynamic or use same width type model
        Conv = []
        Gcn = []
        for i in range(self.pip_num):
            Conv.append(nn.Conv1d(128, 96, kernel_size=3, stride=1, padding=i+1, dilation=i+1))
            Gcn.append(GraphConv(96, 54, norm='none', weight=True, bias=True))
        self.ConvList = nn.ModuleList(Conv)
        self.GcnList = nn.ModuleList(Gcn)


        self.pressConv = nn.Conv1d(self.pip_num, 1, kernel_size=self.pip_num, stride=1, padding=1)

        self.Jk_GIN = nn.ModuleList([
            GraphConv(54, 34, norm='none', weight=True, bias=True), 
            GraphConv(34, 25, norm='none', weight=True, bias=True), 
            GraphConv(25, 16, norm='none', weight=True, bias=True),
            ])
        self.classify1 = nn.Linear(75, 120)
        self.classify2 = nn.Linear(120, self.n_classes)
        self.batchnorm1 = nn.BatchNorm1d(96)
        self.batchnorm2 = nn.BatchNorm1d(54)
        self.batchnorm3 = nn.BatchNorm1d(75)
        self.actf = nn.ReLU()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, graph):
        emd_h = self.BN_embedding(graph)
        emd_h = self.dropout(emd_h)
        logits = self.Presentation(graph, emd_h)
        return logits, emd_h


    def cross_connect(self, h01, h02) -> torch.Tensor:
        h01 = h01.permute(0, 2, 1).squeeze(0)
        return self.batchnorm1(self.actf(h01)+h02.permute(0, 2, 1).squeeze(0))

    # Brain Network embedding
    def BN_embedding(self, graph) -> torch.Tensor:
        # conv1D 需要变形，所以要单独写个函数
        # 普通1D卷积, TCN 不合适
        h0 = graph.ndata['h'].float().unsqueeze(0).permute(0, 2, 1) # shape = [1, 128, 18*n]
        h0x = []
        for i in range(3):
            h0x.append(self.ConvList[i](h0))
        
        edeg_w = graph.edata['f'].float()
        h2x = []
        for i in range(3):
            h1x = self.cross_connect(h0x[i%3], h0x[(i+1)%3])
            temp = self.GcnList[i](graph, h1x, edge_weight=edeg_w)
            h2x.append(self.actf(temp).unsqueeze(0)) # reback to node x emb

        ac_h1 = self.pressConv(torch.cat(h2x, dim=0).permute(
            1, 0, 2)).permute(1, 0, 2).squeeze(0)
        ac_h1 = self.batchnorm2(ac_h1) # without dropout, it's indivual for module 

        return ac_h1

    # presetation module
    def Presentation(self, graph, emd_h) -> torch.Tensor:
        h0x = [emd_h]
        edeg_w = graph.edata['f'].float()
        for i in range(len(self.Jk_GIN)):
            h0x.append(self.actf(self.Jk_GIN[i](graph, h0x[i], edge_weight=edeg_w)))

        with graph.local_scope():
            ac_h = torch.cat(h0x[1:], 1)
            # print(ac_h.shape)
            graph.ndata['h'] = ac_h
            # print(graph)
            hg = dgl.readout_nodes(graph, 'h', None, op='sum', ntype=None)
            return self.classify2(self.classify1(self.batchnorm3(hg)))

    @ property
    def num_labels(self):
        return 5

    def __getitem__(self, idx):
        return self.graphs[idx], self.label[idx]

    def __len__(self):
        return len(self.graphs)


In [13]:
from importlib import reload
import surpport.Args as A
reload(A)
import surpport.mySQL as mySQL
# def prepare():
#     arg = A.Args()
#     model = Classifier4(arg)  # 实际模型
#     # model = Classifier6(arg)
#     base_rcd = mySQL.gen_base_rcd(arg)
#     recorder = {'base': base_rcd}
#     return arg, model, recorder


In [14]:
arg = A.Args()

In [5]:
import surpport.nnstructure as mynn
reload(mynn)

<module 'surpport.nnstructure' from 'e:\\CODEBASE\\myDGL\\FirstDGL\\surpport\\nnstructure.py'>

In [5]:
model = Classifier7(arg)
model = model.cuda()

In [17]:
arg.d_prepare()
arg.m_info(m_name='m4', m_task='220807_test', num=1,)

In [8]:
# def toypy(a, b, c):
#     print(a+b*c)

# args = [1,2,3]
# toypy(*args)

In [18]:
import surpport.dataprocess as DP
import surpport.myfunction as MF
reload(DP)
reload(MF)

<module 'surpport.myfunction' from 'e:\\CODEBASE\\myDGL\\FirstDGL\\surpport\\myfunction.py'>

In [19]:
opt = torch.optim.Adam(model.parameters(), lr=6e-5,
                        eps=1e-8, weight_decay=0.1) 
arg, tr_dataloader, ts_dataloader = DP.dataload(arg, model, opt)

In [23]:
for g, l in tr_dataloader:
    break

In [26]:
nd = g.ndata['h']

In [29]:
nb = nd.view(g.batch_size, 18, -1)
nb.shape

torch.Size([32, 18, 128])

In [11]:
len(arg.tr_id)

1500

In [12]:
base_rcd = mySQL.gen_base_rcd(arg)
recorder = {'base': base_rcd}

In [13]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(arg.tar_path+'\\Journal')

In [20]:
# c7
for epoch in range(1, 10+1):
    print(epoch)
    recorder[str(epoch)+'-th'] = dict()
    rd = recorder[str(epoch)+'-th']
    # 由于图经过拼合，所以需要多一个dataloader的过程
    # 前两个是list
    tr_loss, tr_acc = MF.train(
        epoch,
        model, opt, tr_dataloader,
        arg,
        writer
    )
    mySQL.rcd_log(tr_loss, tr_acc, writer, rd, epoch, 'train')

    el_loss, el_acc, logits, labels = MF.evaluate(
        epoch,
        model, opt, ts_dataloader,
        arg,
        writer
    )

    mySQL.rcd_log(el_loss, el_acc, writer, rd, epoch, 'test')
    mySQL.rcd_result(logits, labels, rd)

    val_acc = np.mean(el_acc, axis=0)
    MF.save_best(val_acc, model, arg)

    mySQL.save_final(epoch, model, val_acc, arg, opt)

mySQL.save_recorder(recorder, arg, 'flow')
print('best acc: %.4f' % (arg.best_acc))
writer.close()

1
Epoch 01, Iter 005,train loss = 1.7542, train acc = 0.1875
Epoch 01, Iter 010,train loss = 1.7813, train acc = 0.1875
Epoch 01, Iter 015,train loss = 1.6585, train acc = 0.2188
Epoch 01, Iter 020,train loss = 1.7175, train acc = 0.1875
Epoch 01, Iter 025,train loss = 1.5395, train acc = 0.2500
Epoch 01, Iter 030,train loss = 1.7440, train acc = 0.1562
Epoch 01, Iter 035,train loss = 1.7914, train acc = 0.1562
Epoch 01, Iter 040,train loss = 1.7855, train acc = 0.0938
Epoch 01, Iter 045,train loss = 1.6683, train acc = 0.2188
Epoch 01, val loss = 1.6796, val acc = 0.2108
未达到期望，未保存模型
2
Epoch 02, Iter 005,train loss = 1.6818, train acc = 0.1250
Epoch 02, Iter 010,train loss = 1.7044, train acc = 0.3125
Epoch 02, Iter 015,train loss = 1.5996, train acc = 0.1875
Epoch 02, Iter 020,train loss = 1.7050, train acc = 0.1250
Epoch 02, Iter 025,train loss = 1.6833, train acc = 0.3125
Epoch 02, Iter 030,train loss = 1.6212, train acc = 0.2500
Epoch 02, Iter 035,train loss = 1.6726, train acc = 0

In [17]:
from surpport.nnstructure import Classifier4
model = Classifier4(arg)
model.cuda()

Classifier4(
  (conv1): Conv1d(128, 96, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(128, 96, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
  (conv3): Conv1d(128, 96, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  (gconv11): GraphConv(in=96, out=54, normalization=none, activation=None)
  (gconv12): GraphConv(in=96, out=54, normalization=none, activation=None)
  (gconv13): GraphConv(in=96, out=54, normalization=none, activation=None)
  (pressConv): Conv1d(3, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (gconv2): GraphConv(in=54, out=25, normalization=none, activation=None)
  (gconv3): GraphConv(in=25, out=13, normalization=none, activation=None)
  (gconv4): GraphConv(in=25, out=11, normalization=none, activation=None)
  (classify1): Linear(in_features=49, out_features=27, bias=True)
  (classify2): Linear(in_features=27, out_features=5, bias=True)
  (actf): ReLU()
  (tanh): Tanh()
  (dropout): Dropout(p=0.2, inplace=False)
  (dropout2):

In [18]:
for epoch in range(1, 10+1):
    print(epoch)
    recorder[str(epoch)+'-th'] = dict()
    rd = recorder[str(epoch)+'-th']
    # 由于图经过拼合，所以需要多一个dataloader的过程
    # 前两个是list
    tr_loss, tr_acc = MF.train(
        epoch,
        model, opt, tr_dataloader,
        arg,
        writer
    )
    mySQL.rcd_log(tr_loss, tr_acc, writer, rd, epoch, 'train')

    el_loss, el_acc, logits, labels = MF.evaluate(
        epoch,
        model, opt, ts_dataloader,
        arg,
        writer
    )

    mySQL.rcd_log(el_loss, el_acc, writer, rd, epoch, 'test')
    mySQL.rcd_result(logits, labels, rd)

    val_acc = np.mean(el_acc, axis=0)
    MF.save_best(val_acc, model, arg)

    mySQL.save_final(epoch, model, val_acc, arg, opt)

mySQL.save_recorder(recorder, arg, 'flow')
print('best acc: %.4f' % (arg.best_acc))
writer.close()

1
Epoch 01, Iter 005,train loss = 452.0110, train acc = 0.3125
Epoch 01, Iter 010,train loss = 520.3890, train acc = 0.1875
Epoch 01, Iter 015,train loss = 506.2921, train acc = 0.2188
Epoch 01, Iter 020,train loss = 649.5613, train acc = 0.3438
Epoch 01, Iter 025,train loss = 588.6839, train acc = 0.2500
Epoch 01, Iter 030,train loss = 494.4493, train acc = 0.1250
Epoch 01, Iter 035,train loss = 534.6677, train acc = 0.2188
Epoch 01, Iter 040,train loss = 570.6909, train acc = 0.2188
Epoch 01, Iter 045,train loss = 507.5078, train acc = 0.2812
Epoch 01, val loss = 560.4825, val acc = 0.1832
未达到期望，未保存模型
2
Epoch 02, Iter 005,train loss = 408.7426, train acc = 0.3125
Epoch 02, Iter 010,train loss = 493.0602, train acc = 0.2188
Epoch 02, Iter 015,train loss = 473.8531, train acc = 0.1562
Epoch 02, Iter 020,train loss = 656.1123, train acc = 0.2188
Epoch 02, Iter 025,train loss = 675.6329, train acc = 0.1875
Epoch 02, Iter 030,train loss = 508.8904, train acc = 0.1875
Epoch 02, Iter 035,tr